In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import re
import time
import os
import warnings
import logging
import io
import csv
import sys
from datetime import datetime
import logging
import scipy

In [4]:
from Local_lib.useful import bs_iv, bs_explicit_call, bs_explicit_put
from Local_lib.models import *
from Local_lib.plot import plot_smiles, plot_points
from Local_lib.volsurf import VolatilitySurface
from Local_lib.volsurf.filters import *
from Local_lib.calibration import calibrate

In [5]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [6]:
DATES = 1000
filters = [filter_delta(0.25), filter_volume(10)]

# Calibration

In [7]:
directory = 'test_run'

In [8]:
# KAPPA, RHO, V0, VBAR, XI, LAMBDA, MU_Y, SIGMA_Y, MU_V
reg = np.array([2e-4, 1e-2, 1e-3, 1e-3, 1e-3, 1e-3, 1e-2, 5e-4, 0])

models = [
    (BlackScholesModel, np.array([1.]), np.diag([0.])),
    (HestonModel, np.array([3., -0.1, 0.8, 0.8, 1.]), np.diag(reg[:5])),
    (SVJModel, np.array([3., -0.1, 0.8, 0.8, 1., 1., 0., 0.1]), np.diag(reg[:8])),
    (SVCJModel, np.array([3., -0.1, 0.8, 0.8, 1., 1., 0., 0.1, 0.01]), np.diag(reg[:9])),
]

"""models = [
    (SVCJModel, np.array([3., -0.1, 0.8, 0.8, 1., 1., 0., 0.1, 0.01]), np.diag(reg[:9]))
]"""

'models = [\n    (SVCJModel, np.array([3., -0.1, 0.8, 0.8, 1., 1., 0., 0.1, 0.01]), np.diag(reg[:9]))\n]'

In [9]:
dates = sorted([item[:8] for item in os.listdir("Data/tickers") if item.endswith('.csv')])
dates = dates[:DATES]
# dates = ["20200204", "20200318"]


In [11]:
def calibrate_day(date, model, x0, reg, filters):
    
    t0 = time.time()
    
    dt = datetime.strptime(date, "%Y%m%d")
    market_vs = VolatilitySurface.from_date(dt, filters=filters)
    calibration_path = f"Results/calibration/results/{directory}/{model.name}/calibration/{date}.csv"

    with open(calibration_path, "w") as stream:
        result = calibrate(model, market_vs, x0, reg, stream=stream)
    
    dt = time.time() - t0
    
    parameters_path = f"Results/calibration/results/{directory}/{model.name}/parameters.csv"

    # Parameters
    with open(parameters_path, 'a+') as f:
        ps = ",".join(["{:.6f}"] * len(result['x'])).format(*result['x'])
        f.write(f"{date},{result['fun']},{result['rmse']},{result['penalty']},{ps}\n")

    errors_path = f"Results/calibration/results/{directory}/{model.name}/errors/{date}.csv"

    # Errors
    model_vs = model.vs(result['x'], market_vs.points)
    
    # create rows
    df1 = model_vs.data.set_index(['ttm', 'moneyness'])
    df2 = market_vs.data.set_index(['', 'moneyness'])
    rows = [[idx[0], idx[1], df1.loc[idx, 'iv'], df2.loc[idx, 'iv']] for idx in df1.index]
    
    # create df
    df = pd.DataFrame(rows, columns=['ttm', 'moneyness', 'model_iv', 'market_iv'])
    df['diff'] = np.abs(df['model_iv'] - df['market_iv'])
    df.to_csv(errors_path, index=False)

    return model.name, date, result['x'], result['fun'], dt

In [41]:
ttm=[]
for date in dates:
        errors_path = f"Results/calibration/results/{directory}/{model.name}/errors/{date}.csv"

        with open(errors_path, 'r') as f:
                ttm.append(pd.read_csv(errors_path)["ttm"])

intervals = [(0, 1/52),(1/52, 2/52),(2/52, 0.25), (0.25, 0.5), (0.5, float('inf'))]

combined_ttm = [ttm_value for ttm_list in ttm for ttm_value in ttm_list]


# Calculate percentages based on the combined ttm list
counts = [sum(interval[0] <= value < interval[1] for value in combined_ttm) for interval in intervals]
percentages = [count / len(combined_ttm) * 100 for count in counts]
percentages = ["{:.2f}%".format(per) for per in percentages]


# Create a DataFrame to display the results across all dates
df = pd.DataFrame({
    'Interval': [f'{interval[0]:.2f} - {interval[1]:.2f}' if interval[1] != float('inf') else f'{interval[0]:.2f}+' for interval in intervals],
    'Percentage': percentages
})
label_mapping = {
    '0.00 - 0.02': '0 to a week',
    '0.02 - 0.04': 'one to two weeks',
    '0.04 - 0.25': 'two weeks to 3 months',
    '0.25 - 0.50': '3 to 6 months',
    '0.50+': '6 months and above',


}
df['Interval'] = result_df['Interval'].map(label_mapping)
df

,Interval,Percentage
0,0 to a week,10.81%
1,one to two weeks,11.43%
2,two weeks to 3 months,30.42%
3,3 to 6 months,27.32%
4,6 months and above,20.01%


In [14]:
with warnings.catch_warnings():
    
    warnings.simplefilter("ignore")
    
    results = []
    
    def callback(retval):
        model_name, date, parameters, rmse, dt = retval
        results.append([model_name, date, rmse, *parameters])
        progress = len(results) / len(dates)
        print(f"\r{model_name:20} {progress:7.2%} {dt:7.2f}s", end='')

    for cls, x, reg in models:
        
        print(f"\r{cls.name:20} {0:7.2%}", end='')

        # take other model parameters as initial value x0
      
        #if cls == HestonModel:
        #    df = pd.read_csv(f'Results/calibration/results/{directory}/BLACK_SCHOLES/parameters.csv')
        #elif cls == SVJModel:
        #    df = pd.read_csv(f'Results/calibration/results/{directory}/HESTON/parameters.csv')
        if cls == SVCJModel:
            df = pd.read_csv(f'Results/calibration/results/{directory}/SVJ/parameters.csv')
                            
        os.makedirs(f"Results/calibration/results/{directory}/{cls.name}/errors", exist_ok=True)
        os.makedirs(f"Results/calibration/results/{directory}/{cls.name}/calibration", exist_ok=True)
        
        parameters_path = f"Results/calibration/results/{directory}/{cls.name}/parameters.csv"
        
        with open(parameters_path, 'w') as f:
            ps = ",".join(cls.labels)
            f.write(f"date,cost,rmse,penalty,{ps}\n")

        all_args = []
            
        for date in dates:
            
            x0 = x[:]
            
            try:
                """if cls == BlackScholesModel:
                    # take min IV for BS
                    dt = datetime.strptime(date, "%Y%m%d")
                    market_vs = VolatilitySurface.from_date(dt, filters=filters)
                    x0 = np.min(market_vs.data['iv'])
                elif cls == HestonModel:
                    # take BS sigma**2 for HESTON parameters V0 and VBAR
                    bs_sigma = df.loc[df['date'] == int(date), 'SIGMA'].iloc[0]
                    x0[2] = x0[3] = bs_sigma ** 2
                elif cls == SVJModel:
                    # take Heston parameters for SVJ
                    heston_parameters = df.loc[df['date'] == int(date), ['KAPPA', 'RHO', 'V0', 'VBAR', 'XI']].iloc[0].to_list()
                    x0[0], x0[1], x0[2], x0[3], x0[4] = heston_parameters
                """
                if cls == SVCJModel:
                    # take SVJ parameters for SVCJ
                    x0[0], x0[1], x0[2], x0[3], x0[4], x0[5], x0[6], x0[7] = df.loc[df['date'] == int(date), ['KAPPA', 'RHO', 'V0', 'VBAR', 'XI', "LAMBDA", "MU_Y", "SIGMA_Y"]].iloc[0].to_list()
            except:
                pass
                        
            model = cls()
            all_args += [[date, model, x0, reg, filters]]

        for args in all_args:
                callback(calibrate_day(*args))
        
        results = []
        
        print("")


SVCJ                  34.72%   27.76s

KeyboardInterrupt: 

In [10]:

dt = datetime(2020, 4, 1)
market_vs = VolatilitySurface.from_date(dt, filters=[filter_delta(0.25), filter_volume(10)])
market_vs.calibrate()
fig = market_vs.plot()

date = datetime.strftime(dt, "%Y%m%d")

models = [BlackScholesModel()]

for model in models:
    x0 = pd.read_csv(f"Results/calibration/results/{directory}/{model.name}/parameters.csv").set_index('date').loc[int(date), list(model.labels)].to_numpy()
    model_vs = model.vs(x0, market_vs.points)
    model_vs.name = model.name
    model_vs.calibrate()
    _ = model_vs.plot(fig=fig)
    print(model_vs - market_vs, dict(zip(model.labels, x0)))


INFO:Local_lib.volsurf.volsurf:>> Calibrating 20200401 SVI volatility surface.


c:\Users\Yassir EL-ORFI\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
c:\Users\Yassir EL-ORFI\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
c:\Users\Yassir EL-ORFI\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_zeros_py.py:479: RuntimeWarning: some failed to converge after 50 iterations
  warnings.warn(msg, RuntimeWarning)


INFO:Local_lib.volsurf.volsurf:Finished.
INFO:Local_lib.volsurf.volsurf:>> Calibrating BLACK_SCHOLES SVI volatility surface.


c:\Users\Yassir EL-ORFI\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
c:\Users\Yassir EL-ORFI\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_zeros_py.py:479: RuntimeWarning: some failed to converge after 50 iterations
  warnings.warn(msg, RuntimeWarning)
c:\Users\Yassir EL-ORFI\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_optimize.py:404: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


INFO:Local_lib.volsurf.volsurf:Finished.
0.06447832081086231 {'SIGMA': 1.033728}
